In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to C:\Users\Hamsaa
[nltk_data]     SK\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to C:\Users\Hamsaa
[nltk_data]     SK\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to C:\Users\Hamsaa
[nltk_data]     SK\AppData\Roaming\nltk_data...


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('D:\Project-ChatBot\Project-ChatBot\intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['How', 'you', 'could', 'help', 'me', '?'], 'options'),
 (['What', 'you', 'can', 'do', '?'], 'options'),
 (['What', 'help', 'you', 'provide', '?'], 'options'),
 (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'),
 (['What', 'support', 'is', 'offered'], 'options'),
 (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'),
 (['Open', 'adverse', 'drugs', 'module'], 'adver

In [11]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('D:\Project-ChatBot\Project-ChatBot\words.pkl','wb'))
pickle.dump(classes,open('D:\Project-ChatBot\Project-ChatBot\classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [12]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Hamsaa SK\AppData\Local\Temp\ipykernel_19792\1296861074.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [13]:
pattern_words

['looking', 'up', 'hospital', 'detail']

In [14]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('D:\Project-ChatBot\Project-ChatBot\chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 908us/step - loss: 2.2107 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1924 - accuracy: 0.1489
Epoch 3/200
10/10 [==============================] - 0s 9ms/step - loss: 2.1002 - accuracy: 0.2340
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0232 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9373 - accuracy: 0.3404
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7598 - accuracy: 0.3830
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.6565 - accuracy: 0.4894
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 1.6279 - accuracy: 0.4043
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.4238 - accuracy: 0.6383
Epoch 10/200
10/10 [==============================] - 0s 1ms/step - loss: 1.1516 - accuracy: 0.6809
Epoch 1

10/10 [==============================] - 0s 860us/step - loss: 0.1008 - accuracy: 0.9787
Epoch 83/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0606 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 897us/step - loss: 0.0161 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 988us/step - loss: 0.0946 - accuracy: 0.9787
Epoch 86/200
10/10 [==============================] - 0s 922us/step - loss: 0.0838 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 705us/step - loss: 0.0742 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 873us/step - loss: 0.1038 - accuracy: 0.9362
Epoch 89/200
10/10 [==============================] - 0s 910us/step - loss: 0.0710 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 810us/step - loss: 0.0750 - accuracy: 0.9787
Epoch 91/200
10/10 [==============================] - 0s 844us/step - loss: 0.0708 - accuracy: 0.

10/10 [==============================] - 0s 819us/step - loss: 0.0472 - accuracy: 0.9787
Epoch 163/200
10/10 [==============================] - 0s 973us/step - loss: 0.0066 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 825us/step - loss: 0.0278 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 827us/step - loss: 0.0315 - accuracy: 0.9787
Epoch 166/200
10/10 [==============================] - 0s 880us/step - loss: 0.0204 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0409 - accuracy: 0.9787
Epoch 168/200
10/10 [==============================] - 0s 998us/step - loss: 0.0162 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 931us/step - loss: 0.0268 - accuracy: 0.9787
Epoch 170/200
10/10 [==============================] - 0s 813us/step - loss: 0.0127 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0534 - accur